In [ ]:
# === nb14_react_multistep_reasoning.ipynb ===
# ReAct 多步推理 (ReAct Multi-step Reasoning)
# 學習目標: 掌握 Reasoning-Acting 循環、工具鏈整合、複雜問題分解

# === Cell 1: Shared Cache Bootstrap ===
import os, torch, platform, pathlib

AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "/mnt/ai/cache")
paths = {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}
for k, v in paths.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)

print("[Cache] Root:", AI_CACHE_ROOT)
print(
    "[GPU]",
    torch.cuda.is_available(),
    torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
)

In [ ]:
# === Cell 2: Install Dependencies ===
import subprocess
import sys


def install_package(package):
    """Install package if not already installed"""
    try:
        __import__(package.split("[")[0])
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])


# Install required packages
packages = [
    "transformers>=4.36.0",
    "accelerate",
    "bitsandbytes",
    "duckduckgo-search",
    "beautifulsoup4",
    "requests",
]

for pkg in packages:
    install_package(pkg)

print("✅ Dependencies installed")

In [ ]:
# === Cell 3: ReAct 核心概念說明 ===
"""
## 🧠 ReAct 推理模式 (Reasoning and Acting)

ReAct 是一種結合推理 (Reasoning) 與行動 (Acting) 的 AI 智能體模式：

### 核心概念 (Core Concepts)
- **Thought (思考)**：AI 分析當前狀況，制定下一步計畫
- **Action (行動)**：執行具體工具調用 (搜尋、計算、檢索等)
- **Observation (觀察)**：獲取行動結果，更新知識狀態
- **Reasoning Chain (推理鏈)**：持續循環直到問題解決

### 與其他模式比較
- **Chain-of-Thought (CoT)**：純思維推理，缺乏外部工具
- **Direct Generation**：直接生成答案，無推理過程
- **ReAct**：思維 + 工具調用，適合複雜多步問題

### 適用場景
✅ 需要外部資訊的問題 (搜尋、計算)
✅ 多步驟分解的複雜任務
✅ 需要驗證與修正的推理過程
❌ 純創意寫作或簡單問答
"""

In [ ]:
# === Cell 4: Basic Tools Definition ===
import requests
from typing import Dict, Any, List, Optional
import json
import re
import math
from datetime import datetime


class SearchTool:
    """Web search tool using DuckDuckGo (no API key required)"""

    def __init__(self):
        self.name = "search"
        self.description = "Search the web for current information"

    def execute(self, query: str) -> str:
        """Execute web search and return results"""
        try:
            from duckduckgo_search import DDGS

            with DDGS() as ddgs:
                results = list(ddgs.text(query, max_results=3))

            if not results:
                return "No search results found"

            formatted_results = []
            for i, result in enumerate(results, 1):
                formatted_results.append(
                    f"{i}. {result['title']}\n"
                    f"   {result['body'][:200]}...\n"
                    f"   Source: {result['href']}\n"
                )

            return "\n".join(formatted_results)

        except Exception as e:
            return f"Search error: {str(e)}"


class CalculatorTool:
    """Mathematical calculation tool"""

    def __init__(self):
        self.name = "calculator"
        self.description = "Perform mathematical calculations"

    def execute(self, expression: str) -> str:
        """Execute mathematical calculation"""
        try:
            # Clean and validate expression
            cleaned = re.sub(r"[^0-9+\-*/.() ]", "", expression)

            # Safe evaluation
            result = eval(cleaned)
            return f"Calculation: {expression} = {result}"

        except Exception as e:
            return f"Calculation error: {str(e)}"


class MemoryTool:
    """Simple memory storage for conversation context"""

    def __init__(self):
        self.name = "memory"
        self.description = "Store and retrieve information"
        self.storage = {}

    def execute(self, action: str, key: str = None, value: str = None) -> str:
        """Execute memory operations (store/retrieve)"""
        try:
            if action == "store" and key and value:
                self.storage[key] = value
                return f"Stored: {key} = {value}"

            elif action == "retrieve" and key:
                if key in self.storage:
                    return f"Retrieved: {key} = {self.storage[key]}"
                else:
                    return f"Key '{key}' not found in memory"

            elif action == "list":
                if self.storage:
                    items = [f"{k}: {v}" for k, v in self.storage.items()]
                    return "Memory contents:\n" + "\n".join(items)
                else:
                    return "Memory is empty"

            else:
                return "Invalid memory operation. Use: store/retrieve/list"

        except Exception as e:
            return f"Memory error: {str(e)}"


# Initialize tools
search_tool = SearchTool()
calc_tool = CalculatorTool()
memory_tool = MemoryTool()

print("🛠️ Tools initialized:")
print(f"- {search_tool.name}: {search_tool.description}")
print(f"- {calc_tool.name}: {calc_tool.description}")
print(f"- {memory_tool.name}: {memory_tool.description}")



In [ ]:
# === Cell 5: LLM Adapter (Lightweight) ===
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


class SimpleLLMAdapter:
    """Lightweight LLM adapter for ReAct reasoning"""

    def __init__(self, model_id: str = "microsoft/DialoGPT-medium"):
        self.model_id = model_id
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        print(f"Loading model: {model_id}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto" if torch.cuda.is_available() else None,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            load_in_4bit=torch.cuda.is_available(),  # Use 4bit if GPU available
        )

        # Add padding token if missing
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def generate(self, prompt: str, max_tokens: int = 256) -> str:
        """Generate response from prompt"""
        try:
            inputs = self.tokenizer.encode(prompt, return_tensors="pt")

            with torch.no_grad():
                outputs = self.model.generate(
                    inputs,
                    max_new_tokens=max_tokens,
                    do_sample=True,
                    temperature=0.7,
                    pad_token_id=self.tokenizer.eos_token_id,
                    attention_mask=torch.ones_like(inputs),
                )

            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract only the new generation
            if prompt in response:
                response = response.replace(prompt, "").strip()

            return response

        except Exception as e:
            return f"Generation error: {str(e)}"


# For demo purposes, we'll use a simple mock LLM
class MockLLM:
    """Mock LLM for demonstration (replace with real LLM in production)"""

    def generate(self, prompt: str, max_tokens: int = 256) -> str:
        """Generate mock responses based on prompt patterns"""
        prompt_lower = prompt.lower()

        if "thought:" in prompt_lower and "search" in prompt_lower:
            return "I need to search for current information about this topic."
        elif "thought:" in prompt_lower and "calculate" in prompt_lower:
            return "I need to perform some mathematical calculations."
        elif "action:" in prompt_lower:
            return "Executing the planned action..."
        else:
            return "Let me think about this step by step."


# Use MockLLM for demo (replace with real LLM)
llm = MockLLM()
print("🤖 LLM adapter initialized (using mock for demo)")

In [ ]:
# === Cell 6: ReAct Agent Core Implementation ===
import re
from typing import Tuple, Optional


class ReActAgent:
    """ReAct (Reasoning + Acting) Agent implementation"""

    def __init__(self, llm, tools: Dict[str, Any], max_iterations: int = 5):
        self.llm = llm
        self.tools = tools
        self.max_iterations = max_iterations
        self.reasoning_chain = []

    def parse_action(self, text: str) -> Tuple[Optional[str], Optional[str]]:
        """Parse action from LLM response"""
        # Look for Action[tool_name]: query pattern
        action_pattern = r"Action\[([^\]]+)\]:\s*(.+)"
        match = re.search(action_pattern, text, re.IGNORECASE)

        if match:
            tool_name = match.group(1).strip()
            query = match.group(2).strip()
            return tool_name, query

        return None, None

    def execute_tool(self, tool_name: str, query: str) -> str:
        """Execute tool with given query"""
        if tool_name not in self.tools:
            return f"Error: Tool '{tool_name}' not found. Available tools: {list(self.tools.keys())}"

        tool = self.tools[tool_name]

        try:
            if tool_name == "memory":
                # Parse memory operations
                parts = query.split(",")
                if len(parts) >= 2:
                    action = parts[0].strip()
                    key = parts[1].strip() if len(parts) > 1 else None
                    value = parts[2].strip() if len(parts) > 2 else None
                    return tool.execute(action, key, value)
                else:
                    return tool.execute(query)
            else:
                return tool.execute(query)

        except Exception as e:
            return f"Tool execution error: {str(e)}"

    def solve(self, question: str) -> Dict[str, Any]:
        """Solve question using ReAct reasoning"""
        self.reasoning_chain = []

        # Initial prompt setup
        system_prompt = """You are an AI assistant that uses ReAct (Reasoning + Acting) to solve problems.

Available tools:
- search: Search the web for information
- calculator: Perform mathematical calculations
- memory: Store and retrieve information (actions: store,key,value or retrieve,key or list)

Format your responses as:
Thought: [Your reasoning about what to do next]
Action[tool_name]: query
Observation: [Tool result will be provided]

Continue this cycle until you can provide a Final Answer.

Example:
Thought: I need to search for information about this topic.
Action[search]: what is the capital of France
Observation: [Search results will appear here]
Thought: Based on the search results, I can now answer.
Final Answer: The capital of France is Paris.
"""

        current_prompt = f"{system_prompt}\n\nQuestion: {question}\n\n"

        for iteration in range(self.max_iterations):
            print(f"\n=== Iteration {iteration + 1} ===")

            # Generate reasoning
            response = self.llm.generate(current_prompt, max_tokens=256)
            print(f"Response: {response}")

            # Record step
            step = {
                "iteration": iteration + 1,
                "prompt": current_prompt,
                "response": response,
                "action": None,
                "observation": None,
            }

            # Check for final answer
            if "final answer:" in response.lower():
                step["final_answer"] = True
                self.reasoning_chain.append(step)

                # Extract final answer
                final_answer_match = re.search(
                    r"final answer:\s*(.+)", response, re.IGNORECASE
                )
                final_answer = (
                    final_answer_match.group(1) if final_answer_match else response
                )

                return {
                    "success": True,
                    "answer": final_answer,
                    "reasoning_chain": self.reasoning_chain,
                    "iterations": iteration + 1,
                }

            # Parse and execute action
            tool_name, query = self.parse_action(response)

            if tool_name and query:
                print(f"Executing: {tool_name}({query})")

                observation = self.execute_tool(tool_name, query)
                print(f"Observation: {observation}")

                step["action"] = {"tool": tool_name, "query": query}
                step["observation"] = observation

                # Update prompt with observation
                current_prompt += f"{response}\nObservation: {observation}\n\n"

            else:
                # No action found, continue with reasoning
                current_prompt += f"{response}\n\n"

            self.reasoning_chain.append(step)

        # Max iterations reached
        return {
            "success": False,
            "answer": "Maximum iterations reached without finding answer",
            "reasoning_chain": self.reasoning_chain,
            "iterations": self.max_iterations,
        }


# Initialize ReAct agent
tools_dict = {"search": search_tool, "calculator": calc_tool, "memory": memory_tool}

react_agent = ReActAgent(llm, tools_dict, max_iterations=5)
print("🧠 ReAct Agent initialized")

In [ ]:
# === Cell 7: Simple Reasoning Example ===
print("=== 簡單推理範例 (Simple Reasoning Example) ===")

# Example 1: Direct calculation
simple_question = "What is 15% of 2,500?"

print(f"Question: {simple_question}")
print("-" * 50)

# Manual ReAct simulation for demo
print("Thought: I need to calculate 15% of 2,500")
print("Action[calculator]: 2500 * 0.15")

calc_result = calc_tool.execute("2500 * 0.15")
print(f"Observation: {calc_result}")

print("Thought: The calculation is complete")
print("Final Answer: 15% of 2,500 is 375")

print("\n✅ Simple reasoning completed")

In [ ]:
# === Cell 8: Multi-step Reasoning Example ===
print("=== 多步推理範例 (Multi-step Reasoning Example) ===")

# Complex question requiring multiple steps
complex_question = "If I invest $10,000 at 3% annual interest compounded annually, how much will I have after 5 years?"

print(f"Question: {complex_question}")
print("-" * 50)

# Step 1: Store initial values
print(
    "Thought: I need to solve a compound interest problem. Let me store the given values first."
)
print("Action[memory]: store,principal,10000")
memory_result1 = memory_tool.execute("store", "principal", "10000")
print(f"Observation: {memory_result1}")

print("Action[memory]: store,rate,0.03")
memory_result2 = memory_tool.execute("store", "rate", "0.03")
print(f"Observation: {memory_result2}")

print("Action[memory]: store,time,5")
memory_result3 = memory_tool.execute("store", "time", "5")
print(f"Observation: {memory_result3}")

# Step 2: Apply compound interest formula
print(
    "\nThought: Now I'll calculate using the compound interest formula: A = P(1 + r)^t"
)
print("Action[calculator]: 10000 * (1.03 ** 5)")
compound_result = calc_tool.execute("10000 * (1.03 ** 5)")
print(f"Observation: {compound_result}")

# Step 3: Calculate interest earned
print("\nThought: Let me also calculate the interest earned")
print("Action[calculator]: 11592.74 - 10000")
interest_result = calc_tool.execute("11592.74 - 10000")
print(f"Observation: {interest_result}")

print("\nThought: I have all the information needed to provide a complete answer")
print(
    "Final Answer: After 5 years, your $10,000 investment at 3% annual compound interest will grow to approximately $11,592.74, earning $1,592.74 in interest."
)

print("\n✅ Multi-step reasoning completed")

In [ ]:
# === Cell 9: Reasoning Chain Visualization ===
def visualize_reasoning_chain(steps: List[Dict], title: str = "Reasoning Chain"):
    """Visualize the reasoning process"""
    print(f"\n📊 {title}")
    print("=" * 60)

    for i, step in enumerate(steps, 1):
        print(f"\n🔹 Step {i}:")

        if "thought" in step:
            print(f"   💭 Thought: {step['thought']}")

        if "action" in step and step["action"]:
            action = step["action"]
            print(f"   🔧 Action: {action['tool']}({action['query']})")

        if "observation" in step and step["observation"]:
            obs = (
                step["observation"][:100] + "..."
                if len(step["observation"]) > 100
                else step["observation"]
            )
            print(f"   👀 Observation: {obs}")

    print("\n" + "=" * 60)


# Simulate a reasoning chain for visualization
demo_chain = [
    {
        "thought": "I need to find current information about this topic",
        "action": {"tool": "search", "query": "latest developments in AI"},
        "observation": "Found 3 recent articles about AI developments...",
    },
    {
        "thought": "Now I need to calculate some values based on the information",
        "action": {"tool": "calculator", "query": "1000 * 1.05"},
        "observation": "Calculation: 1000 * 1.05 = 1050.0",
    },
    {
        "thought": "Let me store this result for later reference",
        "action": {"tool": "memory", "query": "store,result,1050"},
        "observation": "Stored: result = 1050",
    },
]

visualize_reasoning_chain(demo_chain, "示例推理鏈 (Example Reasoning Chain)")

In [ ]:
# === Cell 10: Error Handling and Recovery ===
print("=== 錯誤處理與恢復機制 (Error Handling & Recovery) ===")


class RobustReActAgent(ReActAgent):
    """Enhanced ReAct agent with error handling"""

    def __init__(self, llm, tools: Dict[str, Any], max_iterations: int = 5):
        super().__init__(llm, tools, max_iterations)
        self.error_count = 0
        self.max_errors = 3

    def handle_error(self, error_msg: str, step: int) -> str:
        """Handle errors and suggest recovery actions"""
        self.error_count += 1

        recovery_strategies = [
            "Let me try a different approach to this problem.",
            "I'll reformulate my query and try again.",
            "Let me break this down into smaller steps.",
        ]

        if self.error_count <= self.max_errors:
            strategy = recovery_strategies[
                (self.error_count - 1) % len(recovery_strategies)
            ]
            return f"Error encountered: {error_msg}\nRecovery: {strategy}"
        else:
            return f"Too many errors ({self.error_count}). Unable to continue."

    def execute_tool_safe(self, tool_name: str, query: str) -> str:
        """Execute tool with error handling"""
        try:
            result = self.execute_tool(tool_name, query)

            # Check for error indicators
            if "error" in result.lower():
                return self.handle_error(result, len(self.reasoning_chain))

            return result

        except Exception as e:
            return self.handle_error(str(e), len(self.reasoning_chain))


# Demonstrate error handling
robust_agent = RobustReActAgent(llm, tools_dict)

print("Testing error handling:")
print("1. Invalid tool name")
error_result = robust_agent.execute_tool_safe("invalid_tool", "test query")
print(f"   Result: {error_result}")

print("\n2. Invalid calculation")
error_result2 = robust_agent.execute_tool_safe("calculator", "invalid_expression!!!")
print(f"   Result: {error_result2}")

print("\n✅ Error handling demonstrated")

In [ ]:
# === Cell 11: Comparison of Reasoning Strategies ===
def compare_reasoning_strategies():
    """Compare different reasoning approaches"""

    question = "What's the population of Tokyo and how does it compare to New York?"

    print("=== 推理策略比較 (Reasoning Strategy Comparison) ===")
    print(f"Question: {question}")
    print("-" * 70)

    # 1. Direct Generation
    print("\n🎯 Direct Generation:")
    print(
        "   Response: Tokyo has about 14 million people, while New York has about 8.3 million."
    )
    print("   Pros: Fast, simple")
    print("   Cons: May be outdated, no verification")

    # 2. Chain-of-Thought
    print("\n🔗 Chain-of-Thought:")
    print("   Thought: I need to consider both cities' populations...")
    print("   Thought: Tokyo is the capital of Japan, a major metropolitan area...")
    print("   Thought: New York is the largest city in the US...")
    print("   Response: Based on my knowledge, Tokyo has more people than New York.")
    print("   Pros: Shows reasoning process")
    print("   Cons: Still no external verification")

    # 3. ReAct
    print("\n🧠 ReAct:")
    print("   Thought: I need current population data for both cities")
    print("   Action[search]: Tokyo population 2024")
    print("   Observation: [Current search results]")
    print("   Thought: Now I need New York population data")
    print("   Action[search]: New York population 2024")
    print("   Observation: [Current search results]")
    print("   Thought: Let me calculate the difference")
    print("   Action[calculator]: tokyo_pop - ny_pop")
    print("   Final Answer: [Answer with current, verified data]")
    print("   Pros: Current data, verifiable, transparent process")
    print("   Cons: Slower, requires tools")

    return {
        "direct": {"speed": "fast", "accuracy": "medium", "transparency": "low"},
        "cot": {"speed": "medium", "accuracy": "medium", "transparency": "high"},
        "react": {"speed": "slow", "accuracy": "high", "transparency": "high"},
    }


strategy_comparison = compare_reasoning_strategies()
print(f"\n📊 Strategy Analysis: {strategy_comparison}")

In [ ]:
# === Cell 12: Smoke Test ===
def smoke_test_react():
    """Quick smoke test for ReAct functionality"""
    print("🧪 ReAct Smoke Test")
    print("-" * 30)

    # Test 1: Tool availability
    required_tools = ["search", "calculator", "memory"]
    available_tools = list(tools_dict.keys())

    print("1. Tool Availability:")
    for tool in required_tools:
        status = "✅" if tool in available_tools else "❌"
        print(f"   {status} {tool}")

    # Test 2: Basic tool execution
    print("\n2. Tool Execution:")

    # Calculator test
    calc_test = calc_tool.execute("2 + 2")
    calc_status = "✅" if "4" in calc_test else "❌"
    print(f"   {calc_status} Calculator: {calc_test}")

    # Memory test
    memory_tool.execute("store", "test_key", "test_value")
    memory_test = memory_tool.execute("retrieve", "test_key")
    memory_status = "✅" if "test_value" in memory_test else "❌"
    print(f"   {memory_status} Memory: {memory_test}")

    # Test 3: Action parsing
    print("\n3. Action Parsing:")
    test_response = "Action[calculator]: 10 + 5"
    tool_name, query = react_agent.parse_action(test_response)
    parse_status = "✅" if tool_name == "calculator" and query == "10 + 5" else "❌"
    print(f"   {parse_status} Parse: {tool_name}, {query}")

    # Test 4: LLM basic generation
    print("\n4. LLM Generation:")
    llm_test = llm.generate("Test prompt")
    llm_status = "✅" if llm_test and len(llm_test) > 0 else "❌"
    print(f"   {llm_status} LLM: Generated {len(llm_test)} characters")

    print("\n✅ Smoke test completed")


smoke_test_react()

In [ ]:
# === Cell 13: Usage Examples and Best Practices ===
print("=== 使用範例與最佳實踐 (Usage Examples & Best Practices) ===")

usage_guide = """
## 🎯 ReAct 適用場景 (When to Use ReAct)

✅ **適合使用 ReAct**:
• 需要最新資訊的問題 (股價、新聞、天氣)
• 多步計算或分析任務
• 需要驗證事實的查詢
• 複雜問題分解 (研究報告、比較分析)
• 需要工具輔助的任務 (檔案操作、API 調用)

❌ **不適合使用 ReAct**:
• 簡單創意寫作
• 純邏輯推理問題 (數學證明)
• 即時對話 (延遲敏感)
• 已知答案的基礎問題

## 🛠️ 工具設計原則 (Tool Design Principles)

1. **單一職責**: 每個工具專注一個功能
2. **錯誤處理**: 優雅處理異常情況
3. **明確輸出**: 結構化、可解析的結果
4. **狀態無關**: 工具調用應該是無狀態的
5. **快速響應**: 避免長時間阻塞操作

## ⚡ 性能優化建議 (Performance Tips)

• **限制迭代次數**: 防止無限循環
• **快取結果**: 避免重複工具調用
• **並行執行**: 獨立工具可並行調用
• **提前終止**: 檢測到答案時立即停止
• **輕量級 LLM**: 使用適合任務大小的模型

## 🐛 常見問題與解決方案 (Common Issues)

1. **無限循環**: 設置最大迭代次數，檢測重複模式
2. **工具調用失敗**: 實作重試機制和降級策略
3. **解析錯誤**: 使用更嚴格的格式檢查
4. **記憶體洩漏**: 定期清理推理鏈和工具狀態
5. **回應格式不一致**: 使用結構化提示和格式驗證
"""

print(usage_guide)

In [ ]:
# === Cell 14: Advanced ReAct Patterns ===
print("=== 進階 ReAct 模式 (Advanced ReAct Patterns) ===")


class AdvancedReActAgent(ReActAgent):
    """Enhanced ReAct agent with advanced patterns"""

    def __init__(self, llm, tools: Dict[str, Any], max_iterations: int = 5):
        super().__init__(llm, tools, max_iterations)
        self.context_memory = []
        self.tool_usage_stats = {}

    def add_context(self, context: str):
        """Add contextual information to the agent"""
        self.context_memory.append(context)

    def get_tool_stats(self) -> Dict[str, int]:
        """Get tool usage statistics"""
        return self.tool_usage_stats.copy()

    def execute_tool_with_stats(self, tool_name: str, query: str) -> str:
        """Execute tool and track usage statistics"""
        # Update stats
        self.tool_usage_stats[tool_name] = self.tool_usage_stats.get(tool_name, 0) + 1

        # Execute tool
        result = self.execute_tool(tool_name, query)

        # Log execution
        print(f"   📊 Tool '{tool_name}' used {self.tool_usage_stats[tool_name]} times")

        return result

    def plan_decomposition(self, question: str) -> List[str]:
        """Decompose complex questions into sub-tasks"""
        # Simple heuristic-based decomposition
        if "and" in question.lower():
            parts = question.split(" and ")
            return [part.strip() + "?" for part in parts]
        elif "compare" in question.lower():
            return [
                "Find information about the first item",
                "Find information about the second item",
                "Compare the two items",
            ]
        else:
            return [question]


# Demonstrate advanced patterns
advanced_agent = AdvancedReActAgent(llm, tools_dict)

# Pattern 1: Question Decomposition
complex_q = "What is the population of Tokyo and how does it compare to New York?"
subtasks = advanced_agent.plan_decomposition(complex_q)

print("🔀 Question Decomposition:")
print(f"Original: {complex_q}")
print("Sub-tasks:")
for i, task in enumerate(subtasks, 1):
    print(f"  {i}. {task}")

# Pattern 2: Context-aware reasoning
advanced_agent.add_context("User is interested in urban planning")
advanced_agent.add_context("Previous query was about city demographics")

print(f"\n🧠 Context Memory: {advanced_agent.context_memory}")

# Pattern 3: Tool usage monitoring
print("\n📈 Tool Usage Tracking:")
advanced_agent.execute_tool_with_stats("calculator", "100 + 200")
advanced_agent.execute_tool_with_stats("memory", "store,demo,value")
advanced_agent.execute_tool_with_stats("calculator", "300 * 2")

print(f"Final stats: {advanced_agent.get_tool_stats()}")

In [ ]:
# === Cell 15: Real-world Application Example ===
print("\n=== 實際應用範例 (Real-world Application) ===")


def financial_analysis_demo():
    """Demonstrate ReAct for financial analysis"""

    print("💰 Financial Analysis Scenario")
    print("Question: Should I invest in renewable energy stocks?")
    print("-" * 50)

    # Simulated ReAct reasoning chain
    steps = [
        {
            "step": 1,
            "thought": "I need current information about renewable energy market trends",
            "action": "search: renewable energy stocks performance 2024",
            "observation": "Renewable energy sector showed 15% growth in Q1 2024...",
        },
        {
            "step": 2,
            "thought": "Let me find specific company performances",
            "action": "search: top renewable energy companies stock prices",
            "observation": "Tesla (TSLA): +12%, NextEra Energy (NEE): +8%, Enphase: +20%...",
        },
        {
            "step": 3,
            "thought": "I should calculate potential returns based on different investment amounts",
            "action": "calculator: 10000 * 1.15",
            "observation": "Calculation: 10000 * 1.15 = 11500.0",
        },
        {
            "step": 4,
            "thought": "Let me store key findings for the final recommendation",
            "action": "memory: store,sector_growth,15%",
            "observation": "Stored: sector_growth = 15%",
        },
        {
            "step": 5,
            "thought": "Based on market data and calculations, I can provide a recommendation",
            "action": None,
            "observation": None,
            "final_answer": "Based on current market data showing 15% sector growth and strong individual stock performance, renewable energy stocks appear promising. A $10,000 investment could potentially grow to $11,500 based on sector averages. However, consider diversification and your risk tolerance.",
        },
    ]

    for step_data in steps:
        print(f"\n🔹 Step {step_data['step']}:")
        print(f"   💭 Thought: {step_data['thought']}")

        if step_data["action"]:
            print(f"   🔧 Action: {step_data['action']}")
            print(f"   👀 Observation: {step_data['observation']}")

        if "final_answer" in step_data:
            print(f"   ✅ Final Answer: {step_data['final_answer']}")

    return steps


financial_demo = financial_analysis_demo()


In [ ]:
# === Cell 16: Integration with Other Systems ===
print("\n=== 系統整合範例 (System Integration) ===")


class IntegratedReActAgent(AdvancedReActAgent):
    """ReAct agent with external system integration capabilities"""

    def __init__(self, llm, tools: Dict[str, Any], max_iterations: int = 5):
        super().__init__(llm, tools, max_iterations)
        self.external_apis = {}
        self.data_sources = {}

    def register_api(self, name: str, endpoint: str, headers: Dict = None):
        """Register external API for tool usage"""
        self.external_apis[name] = {"endpoint": endpoint, "headers": headers or {}}
        print(f"📡 Registered API: {name}")

    def register_data_source(self, name: str, source_type: str, config: Dict):
        """Register data source (database, file, etc.)"""
        self.data_sources[name] = {"type": source_type, "config": config}
        print(f"🗄️ Registered data source: {name}")

    def export_reasoning_chain(self, format: str = "json") -> str:
        """Export reasoning chain for external analysis"""
        if format == "json":
            import json

            return json.dumps(self.reasoning_chain, indent=2)
        elif format == "markdown":
            md_content = "# ReAct Reasoning Chain\n\n"
            for i, step in enumerate(self.reasoning_chain, 1):
                md_content += f"## Step {i}\n"
                if "response" in step:
                    md_content += f"**Response:** {step['response']}\n\n"
                if "action" in step and step["action"]:
                    md_content += f"**Action:** {step['action']}\n\n"
                if "observation" in step:
                    md_content += f"**Observation:** {step['observation']}\n\n"
            return md_content
        else:
            return str(self.reasoning_chain)


# Demo integration capabilities
integrated_agent = IntegratedReActAgent(llm, tools_dict)

# Register sample integrations
integrated_agent.register_api("weather_api", "https://api.weather.com/v1/current")
integrated_agent.register_data_source(
    "user_db", "postgresql", {"host": "localhost", "db": "users"}
)

print("Integration setup complete")

In [ ]:
# === Cell 17: Performance Monitoring ===
import time
from typing import Dict, List


class PerformanceMonitor:
    """Monitor ReAct agent performance metrics"""

    def __init__(self):
        self.metrics = {
            "total_queries": 0,
            "successful_queries": 0,
            "average_iterations": 0,
            "average_response_time": 0,
            "tool_usage": {},
            "error_count": 0,
        }
        self.query_logs = []

    def start_query(self, query: str) -> Dict:
        """Start monitoring a query"""
        return {
            "query": query,
            "start_time": time.time(),
            "iterations": 0,
            "tools_used": [],
            "errors": [],
        }

    def log_tool_usage(self, log_entry: Dict, tool_name: str):
        """Log tool usage in current query"""
        log_entry["tools_used"].append(tool_name)
        self.metrics["tool_usage"][tool_name] = (
            self.metrics["tool_usage"].get(tool_name, 0) + 1
        )

    def log_error(self, log_entry: Dict, error: str):
        """Log error in current query"""
        log_entry["errors"].append(error)
        self.metrics["error_count"] += 1

    def finish_query(self, log_entry: Dict, success: bool):
        """Finish monitoring a query and update metrics"""
        log_entry["end_time"] = time.time()
        log_entry["duration"] = log_entry["end_time"] - log_entry["start_time"]
        log_entry["success"] = success

        self.query_logs.append(log_entry)
        self.metrics["total_queries"] += 1

        if success:
            self.metrics["successful_queries"] += 1

        # Update averages
        self._update_averages()

    def _update_averages(self):
        """Update average metrics"""
        if self.query_logs:
            total_iterations = sum(log.get("iterations", 0) for log in self.query_logs)
            total_time = sum(log.get("duration", 0) for log in self.query_logs)

            self.metrics["average_iterations"] = total_iterations / len(self.query_logs)
            self.metrics["average_response_time"] = total_time / len(self.query_logs)

    def get_report(self) -> str:
        """Generate performance report"""
        success_rate = (
            (self.metrics["successful_queries"] / self.metrics["total_queries"] * 100)
            if self.metrics["total_queries"] > 0
            else 0
        )

        report = f"""
📊 ReAct Performance Report
==========================
Total Queries: {self.metrics['total_queries']}
Successful: {self.metrics['successful_queries']} ({success_rate:.1f}%)
Average Iterations: {self.metrics['average_iterations']:.1f}
Average Response Time: {self.metrics['average_response_time']:.2f}s
Errors: {self.metrics['error_count']}

Tool Usage:
{chr(10).join(f'  {tool}: {count}' for tool, count in self.metrics['tool_usage'].items())}
"""
        return report


# Demo performance monitoring
monitor = PerformanceMonitor()

# Simulate some queries
for i in range(3):
    log = monitor.start_query(f"Sample query {i+1}")
    log["iterations"] = i + 2
    monitor.log_tool_usage(log, "search")
    monitor.log_tool_usage(log, "calculator")
    if i == 2:  # Simulate error in last query
        monitor.log_error(log, "Tool timeout")
    monitor.finish_query(log, success=(i != 2))
    time.sleep(0.1)  # Small delay for realistic timing

print(monitor.get_report())

In [ ]:
# === Cell 18: Best Practices Summary ===
print("=== 最佳實踐總結 (Best Practices Summary) ===")

best_practices = {
    "設計原則 (Design Principles)": [
        "保持工具功能單一且明確",
        "實作健全的錯誤處理機制",
        "使用結構化的提示格式",
        "限制最大迭代次數防止無限循環",
        "提供清晰的工具使用說明",
    ],
    "性能優化 (Performance Optimization)": [
        "快取常用查詢結果",
        "並行執行獨立工具調用",
        "使用輕量級模型進行簡單任務",
        "實作智能提前終止機制",
        "監控和分析工具使用模式",
    ],
    "可靠性 (Reliability)": [
        "實作重試機制處理暫時性錯誤",
        "提供降級策略當主要工具失效",
        "驗證工具輸出格式和內容",
        "記錄詳細的執行日誌",
        "定期測試所有工具功能",
    ],
    "可維護性 (Maintainability)": [
        "模組化工具設計便於擴展",
        "使用配置文件管理工具參數",
        "提供完整的文檔和範例",
        "實作自動化測試套件",
        "版本控制工具和提示模板",
    ],
}

for category, practices in best_practices.items():
    print(f"\n🎯 {category}:")
    for practice in practices:
        print(f"   • {practice}")

In [ ]:
# === Cell 19: Final Smoke Test ===
def comprehensive_smoke_test():
    """Comprehensive smoke test for all ReAct components"""
    print("🧪 綜合測試 (Comprehensive Smoke Test)")
    print("=" * 50)

    test_results = {}

    # Test 1: Basic ReAct functionality
    try:
        basic_agent = ReActAgent(llm, tools_dict, max_iterations=2)
        test_results["basic_agent"] = "✅ PASS"
    except Exception as e:
        test_results["basic_agent"] = f"❌ FAIL: {e}"

    # Test 2: Advanced agent features
    try:
        adv_agent = AdvancedReActAgent(llm, tools_dict)
        adv_agent.add_context("test context")
        decomp = adv_agent.plan_decomposition("test question")
        test_results["advanced_features"] = "✅ PASS"
    except Exception as e:
        test_results["advanced_features"] = f"❌ FAIL: {e}"

    # Test 3: Performance monitoring
    try:
        perf_monitor = PerformanceMonitor()
        log = perf_monitor.start_query("test")
        perf_monitor.finish_query(log, True)
        report = perf_monitor.get_report()
        test_results["performance_monitoring"] = "✅ PASS"
    except Exception as e:
        test_results["performance_monitoring"] = f"❌ FAIL: {e}"

    # Test 4: Integration capabilities
    try:
        int_agent = IntegratedReActAgent(llm, tools_dict)
        int_agent.register_api("test_api", "http://test.com")
        export = int_agent.export_reasoning_chain("json")
        test_results["integration"] = "✅ PASS"
    except Exception as e:
        test_results["integration"] = f"❌ FAIL: {e}"

    # Test 5: All tools functional
    tool_tests = {}
    for tool_name, tool in tools_dict.items():
        try:
            if tool_name == "calculator":
                result = tool.execute("1 + 1")
                tool_tests[tool_name] = "✅ PASS" if "2" in result else "❌ FAIL"
            elif tool_name == "memory":
                tool.execute("store", "test", "value")
                result = tool.execute("retrieve", "test")
                tool_tests[tool_name] = "✅ PASS" if "value" in result else "❌ FAIL"
            else:
                # For search tool, just check it doesn't crash
                result = tool.execute("test")
                tool_tests[tool_name] = "✅ PASS"
        except Exception as e:
            tool_tests[tool_name] = f"❌ FAIL: {e}"

    # Print results
    print("Core Components:")
    for test_name, result in test_results.items():
        print(f"  {result} {test_name}")

    print("\nTool Tests:")
    for tool_name, result in tool_tests.items():
        print(f"  {result} {tool_name}")

    # Overall status
    all_passed = all(
        "✅" in result for result in {**test_results, **tool_tests}.values()
    )
    overall_status = "✅ ALL TESTS PASSED" if all_passed else "⚠️ SOME TESTS FAILED"
    print(f"\n{overall_status}")

    return {"core": test_results, "tools": tool_tests, "overall": all_passed}


final_test_results = comprehensive_smoke_test()

print("\n" + "=" * 60)
print("📝 Notebook nb14_react_multistep_reasoning.ipynb completed!")
print("✅ Ready for Git commit and next stage planning")


## 6. 本章小結

### 🎯 完成項目 (Completed Items)
- **ReAct 核心引擎**：實作完整的 Reasoning-Acting 循環機制
- **工具生態系統**：搜尋、計算、記憶三大基礎工具
- **多步推理能力**：複雜問題分解與逐步解決
- **錯誤恢復機制**：處理工具失效與推理中斷
- **性能監控**：追蹤執行效率與工具使用統計
- **可視化界面**：推理鏈展示與決策過程透明化

### 🧠 核心概念要點 (Key Concepts)
- **Thought-Action-Observation 循環**：ReAct 的核心運作模式
- **工具抽象化**：統一的工具接口設計原則
- **推理策略比較**：ReAct vs CoT vs Direct 的適用場景
- **錯誤處理策略**：重試、降級、提前終止機制
- **性能優化技巧**：快取、並行、智能終止

### ⚠️ 常見陷阱 (Common Pitfalls)
- **無限循環風險**：必須設置最大迭代次數限制
- **工具調用格式**：需要嚴格的解析規則避免格式錯誤
- **上下文長度**：推理鏈過長可能超出模型限制
- **工具可靠性**：外部API可能失效需要降級策略
- **成本控制**：多步推理會增加 token 消耗

### 🚀 下一步建議 (Next Steps)

**立即可行的擴展**：
1. **更多工具類型**：檔案操作、資料庫查詢、API 調用
2. **智能工具選擇**：根據問題類型自動選擇最佳工具組合
3. **並行推理**：獨立子任務的並行執行

**整合其他 Notebook**：
- **nb13 (Function Calling)**：整合更豐富的工具庫
- **nb26 (RAG Basic)**：結合檢索增強生成
- **nb29 (Multi-Agent)**：多代理協作推理

**建議下一個重點**：
建議先完成 **nb29 Multi-Agent Collaboration**，因為：
- ReAct 為單一代理推理，多代理是自然延伸
- 可重用本章的工具和推理框架
- 為後續的自動化流程 (nb30) 打下基礎

準備好繼續下一個 Notebook 的開發！